# The purpose of this program is to create safety scores, safety ratings, and accident information for crashes in Melbourne


##  Reading in data and preprocessing

In [48]:
library(reshape2)
library(dplyr)
library(hash)

In [49]:
crashes=read.csv("Crashes_Last_Five_Years.csv", stringsAsFactors = FALSE)
loc = read.csv("ACCIDENT_LOCATION.csv", stringsAsFactors = FALSE)

crashes = Reduce(function(x,y) merge(x = x, y = y, by = "ACCIDENT_NO"), 
       list(loc,crashes))

To get the safety rating of each road, we look at the total cost to society, including average cost of damge to vehicles, cost of a fatality, cost of a serious injury and so on. All figures were sourced from the article below:

https://www.aaa.asn.au/wp-content/uploads/2018/03/AAA-National-Road-Safety-Platform_Sep-2017.pdf

In [50]:
# get cost of crashes
crashes$FATALITY = crashes$FATALITY*4.339
crashes$SERIOUSINJURY = crashes$SERIOUSINJURY*0.239
crashes$OTHERINJURY = crashes$OTHERINJURY*0.012
crashes$NONINJURED = 0.003*crashes$NO_OF_VEHICLES
crashes$COST = crashes$FATALITY + crashes$SERIOUSINJURY + crashes$OTHERINJURY + crashes$NONINJURED

names(crashes) = tolower(names(crashes))

# set year for ratings
ratings_year = 2018

In [51]:
# get the roads for accidents
crashes$road = (paste(crashes$road_name,crashes$road_name_int, sep = ","))
crashes$road = (trimws(crashes$road))
crashes = crashes[crashes$road!=",",] 

# get accident roads for meblourne
melbs = crashes[crashes$deg_urban_name %in% c('MELB_URBAN', 'MELBOURNE_CBD'),]

In [52]:
# get year of crashes
get_year= function (value) {
    name = as.character(value)
    digits = unlist(strsplit(name, "/"))
    return (as.numeric(digits[3]))
}

crashes$year = as.numeric(unlist(lapply(crashes$accident_date,get_year)))


## Scoring and Rating

With all the preprocessing finished, we have to get the costs for each road in 2018. WWe use these score roads (1-100), and assign a rating (1-3).

In [53]:
# get road costs by year
yearly = crashes %>% group_by(road,year) %>% summarise(cost =sum(cost))

# get road costs in 2018 only
x2018 = yearly %>% filter (year==ratings_year) 

# for other roads which did not have a crash in 2018, we add them, and assign a cost of zero
roads =  crashes %>% group_by(road)%>% summarise(cost = 0)
all2018 = merge(roads,x2018, by=c("road"),all.x=TRUE)
all2018[is.na(all2018$cost.y),"cost.y"]=0

all2018$year = ratings_year

To assign a score to each road, we find the quantile of each cost greater than zero and multiply by 100. 
Costs of zero are automatically assigned a score of 1.

In [54]:
riskier = all2018[(all2018$cost.y>0),]

get_percentile = ecdf(riskier$cost.y)
assign_score = function (cost){
    value = get_percentile(cost)
    value = if (value == 0) 1 else value*100
    
    return (value)
}

all2018$score = as.numeric(unlist(lapply(all2018$cost.y,assign_score)))

Once roads, are scored we can assign a rating. We decided to use 70 and 95 as boundaries for the ratings.

In [55]:
get_rating = function (score){
   
    val = 3

    if (score<=70){
        val = 1

    }
    if (score >70 & score<=95){
        val = 2
    }
            
    return (val)
}
all2018$rating = as.numeric(unlist(lapply(all2018$score,get_rating)))

# get only roads in melbourne
melbs = merge(melbs,all2018,by = c('road'), all.x = TRUE)

In [59]:
names(melbs)

[1] "road"               "accident_no"        "node_id.x"         
 [4] "road_route_1"       "road_name"          "road_type"         
 [7] "road_name_int"      "road_type_int"      "distance_location" 
[10] "direction_location" "nearest_km_post"    "off_road_location" 
[13] "x"                  "y"                  "objectid"          
[16] "abs_code"           "accident_status"    "accident_date"     
[19] "accident_time"      "alcoholtime"        "accident_type"     
[22] "day_of_week"        "dca_code"           "hit_run_flag"      
[25] "light_condition"    "police_attend"      "road_geometry"     
[28] "severity"           "speed_zone"         "run_offroad"       
[31] "node_id.y"          "longitude"          "latitude"          
[34] "node_type"          "lga_name"           "region_name"       
[37] "vicgrid_x"          "vicgrid_y"          "total_persons"     
[40] "inj_or_fatal"       "fatality"           "seriousinjury"     
[43] "otherinjury"        "noninjured"         "males"             
[46] "females"            "bicyclist"          "passenger"         
[49] "driver"             "pedestrian"         "pillion"           
[52] "motorist"           "unknown"            "ped_cyclist_5_12"  
[55] "ped_cyclist_13_18"  "old_pedestrian"     "old_driver"        
[58] "young_driver"       "alcohol_related"    "unlicencsed"       
[61] "no_of_vehicles"     "heavyvehicle"       "passengervehicle"  
[64] "motorcycle"         "publicvehicle"      "deg_urban_name"    
[67] "deg_urban_all"      "lga_name_all"       "region_name_all"   
[70] "srns"               "srns_all"           "rma"               
[73] "rma_all"            "divided"            "divided_all"       
[76] "stat_div_name"      "cost"               "cost.x"            
[79] "year"               "cost.y"             "score"             
[82] "rating"

##  Voice Alert Information

For voice alerts, we provide drive with the accident type description and road geometry.

In [62]:
# melbs data for 2018 only
current = melbs[melbs$year==ratings_year,]

# get the most frequent descriptions and road geometry per road
accident_type_info = current %>% group_by(road) %>% count(accident_type) %>% top_n(1) %>% distinct(road,n, .keep_all= TRUE) %>% select(c(road,accident_type))
road_geom_info = current %>% group_by(road) %>% count(road_geometry) %>% top_n(1) %>% distinct(road,n, .keep_all= TRUE) %>% select(c(road,road_geometry))

Selecting by n
Selecting by n


In [64]:
# create a dictionary of road to info and road geometry
create_dict = function(df){
    h = hash() 
    for (i in 1:nrow(df)){
        key = df[i,1][[1]]
        val = df[i,2][[1]]
        h[[key]] = val
    }
    return(h)
} 
accident_type_dict = create_dict(accident_type_info)
road_geom_dict = create_dict(road_geom_info)

In [65]:
# create the information for voice alerts
create_voice_info = function(road){  
    accident_type = accident_type_dict[[road]]
    road_geom = road_geom_dict[[road]]
   
    
    
    if (accident_type == "Collision with vehicle"){
        accident_type = "high vehicle collisions"
    }
    else if (accident_type == "Struck Pedestrian"){
        accident_type = "high struck pedestrians"
    }
    else if (accident_type == "Struck animal"){
        accident_type = "high struck animals"
    }
    else if (accident_type == "Collision with a fixed object"){
        accident_type = "high fixed object collision"
    }
    else{
        accident_type = "high risk"
    }
    
    if (road_geom %in% c("Not at intersection", "Unknown", "Road closure", "Road closure'")){
        road_geom = "road"
    }
    return ( toupper(paste(accident_type, "at" , road_geom)) )
}

current$voice_info = unlist(lapply(current$road,create_voice_info))
road_acc_info_2018 = current %>% distinct(road,voice_info) 

In [66]:
# get voice info for all roads in melbourne. roads without crashes have "None" as information
new_all = merge(melbs,road_acc_info_2018,by =  c('road'), all.x = TRUE)
new_all[is.na(new_all$voice_info),"voice_info"]="None"

data = new_all[,c("rating","score","voice_info","x","y")]

# risky roads which were missing info are assigned
data[data$voice_info=="None" & data$rating==3,"voice_info"] = "high risk at road"

In [70]:
# write data
write.table(data, file = "safety_data.csv", sep=",",  col.names=FALSE, row.names = FALSE)

------